<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/01.tned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/01.tned.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

# Text Normalization, Edit Distance

📝 SALP chapter 2

## What is Sentence Segmentation?
- **Sentence Segmentation** is the process of dividing a corpus (a large collection of text) into individual sentences.
  - Typically, sentences are separated by **punctuation marks** such as periods (.), exclamation marks (!), or question marks (?).
- Challenges
  - **Abbreviations:** e.g., "Dr.", "etc.", "Mr." could be mistaken as sentence boundaries.
  - **Quotes and Parentheses:** Sentences within quotes or parentheses can create ambiguity.
  - **Ellipses:** Ellipsis (...) can be mistaken as multiple sentence boundaries.

- 🍎 Segment text into sentences

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

text = "Mr. Smith went to Washington. He enjoyed his time there."
sentences = sent_tokenize(text)
print(sentences)

## Segmenting Sentences into Words
- **Word Tokenization** is the process of breaking down a sentence into individual words or tokens.
- Word tokenization helps in identifying the `basic units of meaning` in a text.
- Challenges
  - **Contractions:** e.g., "don't" can be split into "do" and "n't".
  - **Hyphenated Words:** e.g., "state-of-the-art" could be considered one word or multiple words.
- 🍎 segment a sentence into words

In [ ]:
from nltk.tokenize import word_tokenize

sentence = "Mr. Smith went to Washington."
words = word_tokenize(sentence)
print(words)

## Punctuation in Segmentation
- **Sentence Segmentation:** Periods, exclamation marks, and question marks usually denote the end of a sentence.
- **Word Tokenization:** Punctuation marks like commas, semicolons, and quotes are often separated from the words they are attached to.
- 🍎
  - Sentence: "Hello, world!"
  - Tokenized: ["Hello", ",", "world", "!"]
- Whether to keep or discard punctuation depends on the task. 
  - For sentiment analysis, punctuation can carry important emotional context.

## Spoken Language Components
- **Utterances:** A complete unit of speech that may not be grammatically correct.
- **Disfluencies:** Interruptions in the flow of speech, e.g., "uh," "um," "you know."
- **Fragments:** Incomplete sentences or phrases.
- **Fillers:** Words or sounds that fill gaps in speech but do not add meaning, e.g., "like," "you know."
- **Filled Pauses:** Prolonged sounds such as "uhhh" or "ummm."
- 🍎
  - Utterance: "Well, I... I think, um, it's great!"
  - Tokenized: ["Well", ",", "I", "...", "I", "think", ",", "um", ",", "it's", "great", "!"]
- Disfluencies may signal that the speaker is restarting the clause or idea
  - helpful in speech recognition in predicting the upcoming word
  - treated as regular words in such cases

## Word Types vs. Word Instances
- Word Types
  - **Word Types** refer to distinct words in a text, ignoring repetition.
  - 🍎: In the text "cat cat dog," "cat" and "dog" are the word types.
- Word Instances
  - **Word Instances** refer to the total occurrences of words in a text.
  - 🍎: In the text "cat cat dog," there are 3 word instances: two "cat" and one "dog."

## Case Sensitivity in Word Types
- **Case Sensitivity:** 
  - In some NLP tasks, "Word" and "word" may be considered different types.
  - This distinction is important in cases where capitalization changes meaning
    - e.g., "Apple" (company) vs. "apple" (fruit).
- **Normalization:** 
  - Often, texts are normalized to lowercase to treat "Word" and "word" as the same type for tasks like `word frequency analysis`.

- 🍎 case sensitivity

In [ ]:
words = ["Word", "word", "Apple", "apple"]
lowercase_words = [word.lower() for word in words]
print(set(lowercase_words)) 

| **Corpus Name**       | **Word Instances** | **Word Types**  | **Description**                                                                 |
|-----------------------|--------------------|-----------------|---------------------------------------------------------------------------------|
| **Brown Corpus**      | ~1,000,000         | ~50,000         | The first million-word electronic corpus of American English.                   |
| **Penn Treebank**     | ~4,500,000         | ~100,000        | Annotated corpus of English, primarily used for training NLP models.            |
| **British National Corpus (BNC)** | ~100,000,000      | ~600,000        | A wide range of modern British English (spoken and written).                    |
| **COCA (Corpus of Contemporary American English)** | ~1,000,000,000   | ~450,000        | Largest freely available corpus of American English, covering 1990-2019.        |
| **Google Ngrams**     | ~500,000,000,000   | ~13,588,391     | Dataset of n-grams (1 to 5 words) from scanned books dating from 1500-2008.     |
| **Reuters Corpus**    | ~810,000           | ~30,000         | Collection of news documents used for text classification and NLP research.     |
| **Wikipedia Corpus**  | ~1,000,000,000+    | ~20,000,000+    | Continuously updated corpus of Wikipedia articles, covering a vast range of topics. |
| **Gutenberg Corpus**  | ~25,000,000        | ~200,000        | Collection of classic literature and historical texts available as eBooks.      |

## 🏃 Explore [Google N-grams Corpus](https://books.google.com/ngrams/info) 
- an extensive dataset consisting of n-grams (sequences of 1 to 5 words) derived from a large corpus of books scanned by Google
- contain around 500 billion word instances, over 13 million unique word types
- [Google books syntactic N-grams](https://commondatastorage.googleapis.com/books/syntactic-ngrams/index.html)

## Herdan’s Law (Heaps’ Law)
- crucial for predicting vocabulary growth in linguistics and NLP
- as a text corpus grows, vocabulary size increases at a diminishing rate
- **Formal Definition:**
  - the vocabulary size $V$ grows as a sublinear power-law function of the number of word tokens $N$ in the corpus:
  - $ V(N) = k \times N^\beta $
    - $ V(N) $ is the number of distinct word types (vocabulary size).
    - $ N $ is the total number of word tokens.
    - $ k $ is a constant that depends on the corpus.
    - $ 0 < \beta <1 $ is an exponent that usually ranges between 0.4 and 0.6.
-  **Key Points:**
   - **Sublinear Growth**: 
     - As more words are added to the corpus, the rate of new word types (unique words) discovered decreases. 
     - Initially, the vocabulary grows quickly, but over time, as more text is added, the discovery of new unique words slows down.
   - **Vocabulary Saturation**: 
     - In any given language, there is a finite number of commonly used words. 
     - As the corpus size increases, fewer and fewer new words are encountered, and the vocabulary growth rate diminishes.
   - **Implications for Language Models**: 
     - Even with large corpora, the vocabulary size will not grow indefinitely and will eventually plateau.

## Wordform and Lemma
- A **wordform** is the specific written or spoken version of a word
  - including its tense, case, number, or mood, depending on the language.
  - 🍎
    - *Runs, running, ran* are different wordforms of the verb *run*.
    - *Cats, cat's, cats'* are different wordforms of the noun *cat*
- A **lemma** is the canonical or dictionary form of a word
  - representing all its possible wordforms.
  - the base or root form used to look up words in dictionaries.
  - often used in linguistic analysis to group different wordforms together.
  - 🍎
    - The lemma for the wordforms *runs, running, ran* is *run*.
    - The lemma for the wordforms *cats, cat's, cats'* is *cat*.
- The process of converting a wordform into its lemma is called **lemmatization**

## Unix Tools for Word Tokenization
- `tr` (translate): Convert or delete characters in text.
- `cut`: Remove sections from each line of files or input.
- `awk`: Pattern scanning and processing language.
- `grep`: Search for patterns in text.
  - 🍎 Tokenizing a text file
    ```bash
    grep -oE '\w+' input.txt > tokens.txt
    ```

In [ ]:
%%bash
# 1. `tr` replaces spaces with newlines
echo "Hello, world! How are you?" | tr -s ' ' '\n'

In [ ]:
%%bash
# 2. `cut` splits the text using a space as a delimiter and extracts the first word.
echo "Hello, world! How are you?" | cut -d ' ' -f 1

In [ ]:
%%bash
# 3. `awk` prints each word separately by iterating over fields (`$i`)
echo "Hello, world! How are you?" | awk '{for(i=1;i<=NF;i++) print $i}'

In [ ]:
%%bash
# 4. `grep` finds and prints each word matching the regex pattern `\w+` (alphanumeric characters).
echo "Hello, world! How are you?" | grep -oE '\w+'

In [ ]:
%%bash
# 5. Combining `tr` and `awk`
# - `tr -s ' ' '\n'` tokenizes by spaces.
# - `awk '{print tolower($0)}'` converts each token to lowercase.

echo "Hello, world! How are you?" | tr -s ' ' '\n' | awk '{print tolower($0)}'

## 🏃 Practice

In [ ]:
%%bash
# 0. Download [Shakespeare's](https://www.gutenberg.org/cache/epub/100/pg100.txt) works in a text file, save it as `sh.txt`
wget -O sh.txt https://www.gutenberg.org/cache/epub/100/pg100.txt 
# 1. ‘squeezed’ a series of non-alphabetic characters in a row into a single newline
tr -sc 'A-Za-z' '\n' < sh.txt # sh.txt contains all Shakespeare's works
# 2. find the vocabulary of Shakespeare's works
tr -sc 'A-Za-z' '\n' < sh.txt | sort | uniq -c
# 3. convert Shakespeare's vocabulary into lowercase
tr -sc 'A-Za-z' '\n' < sh.txt | tr A-Z a-z | sort | uniq -c
# 4. find frequent words
tr -sc 'A-Za-z' '\n' < sh.txt | tr A-Z a-z | sort | uniq -c | sort -n -r > corpus.txt


## Tokenization Methods in NLP
- Top-down (Rule-based) Tokenization
  - Uses predefined rules to split text into tokens
  - Typically faster but less flexible than machine learning approaches
  - Penn Treebank Tokenization
    - Developed for the [Penn Treebank Project](https://catalog.ldc.upenn.edu/LDC99T42)
    - Specific rules for English text

🍎

In [ ]:
from nltk.tokenize import word_tokenize

text = "Don't hesitate to email john.doe@example.com"
tokens = word_tokenize(text)
print(tokens)


### 🍎 NLTK RegexpTokenizer
- Uses regular expressions for flexible tokenization

In [ ]:
from nltk.tokenize import RegexpTokenizer

text = "Hello, world! How's it going?"
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
print(tokens)

## Chinese, Korean, and Japanese Tokenization

- These languages don't use spaces between words
- Requires specialized algorithms
- Chinese Tokenization
  - Uses techniques like maximum matching or statistical models
- Korean Tokenization
  - Considers complex morphological structure
- Japanese Tokenization
  - Uses dictionary-based approaches or statistical models

In [ ]:
# Check if jieba is installed on Google Colab
import sys
in_colab = 'google.colab' in sys.modules
jieba_installed = 'jieba' in sys.modules

if in_colab and not jieba_installed:
    print("jieba is not installed. Installing now...")
    %pip install jieba

In [ ]:
# 🍎 1. Tokenize Chinese using jieba library

import jieba

text = "我喜欢自然语言处理"
tokens = jieba.cut(text)
print(list(tokens))

## Bottom-up Tokenization Algorithms
- also called Subword tokenization methods 
- efficiently handle rare words and out-of-vocabulary terms
- [Byte-Pair Encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding)
  - Merges the most frequent pair of bytes or characters until a desired vocabulary size is reached
  - 🍎 Word `"lower"` could be split into `"low"` + `"er"` if `"lower"` is rare but `"low"` and `"er"` are common.
- `Unigram Language Modeling (SentencePiece)`
  - Instead of merging, it starts with a large set of potential subword units.
  - Find the subset of subword units that maximizes the likelihood of the training data.

###  **Comparison: BPE vs. Unigram Language Modeling**
- **BPE**:
  - **Deterministic**: Iteratively merges most frequent pairs.
  - **Example**: `"lowered"` → `"low"` + `"ered"`.

- **Unigram**:
  - **Probabilistic**: Selects subwords to maximize likelihood.
  - **Example**: `"lowered"` → `"low"` + `"ered"` based on likelihood.

In [ ]:
# 🍎 1. BPE in Python

from tokenizers import Tokenizer, models, trainers

tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=5000)
tokenizer.train(["corpus.txt"], trainer)
encoded = tokenizer.encode("lowered")
print(encoded.tokens)

In [ ]:
# 🍎 2. Unigram in Python

import sentencepiece as spm

spm.SentencePieceTrainer.train(input='corpus.txt', model_prefix='m', vocab_size=5000, model_type='unigram')
sp = spm.SentencePieceProcessor(model_file='m.model')
print(sp.encode_as_pieces("lowered"))



## ⚠️ In neural networks
  - Words are `NOT` used as the internal unit of representation at all! 
  - The input strings are tokenized into words even `only parts` of words.

## Byte-Pair Encoding (BPE)

- Initially developed as a text compression algorithm.
- Adopted by OpenAI for tokenization in the GPT models.
- used in GPT, GPT-2, RoBERTa, BART, DeBERTa.

---

### **The BPE Training Algorithm**

- **Step 1**: Compute the unique set of words in the corpus after normalization and pre-tokenization.
- **Step 2**: Build the initial vocabulary from all symbols used in these words.
- **Example Corpus**: "hug", "pug", "pun", "bun", "hugs".
- **Initial Vocabulary**: ["b", "g", "h", "n", "p", "s", "u"].
- **Real-World Vocabulary**: ASCII characters and possibly Unicode.

---

### **Byte-Level BPE**

- **Purpose**: Handles all characters by using bytes instead of Unicode.
- **Base Vocabulary Size**: 256 (for all byte values).
- **Advantage**: Avoids unknown tokens for unseen characters.

---

### **BPE Merge Process**

- **Step 1**: Identify the most frequent pair of tokens in the corpus.
- **Step 2**: Merge the pair into a new token.
- **Repeat**: Continue until the desired vocabulary size is reached.

---

### 🍎 **Example of BPE Merging**

- **Corpus Frequencies (word, frequency)**:
  - ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
  - ("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
- **Merge 1**: Identify and merge the most frequent pair.
  - **First Pair**: (("u", "g)", 20) → ("ug", 20).
  - **Updated Vocabulary**: ["b", "g", "h", "n", "p", "s", "u", "ug"].
  - **Updated Corpus**: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
- **Merge 2**: Identify next most frequent pair.
  - **Second Pair**: (("u", "n"), 16) → "un".
  - **Updated Vocabulary**: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"].
  - **Updated Corpus**: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
- **Merge 3**: Continue merging until vocabulary size is reached.
  - **Next Pair**: (("h", "ug"), 15) → "hug".
  - **Updated Vocabulary**: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"].
  - **Updated Corpus**: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
- **Continue like this** until we reach the desired vocabulary size.

### Tokenization Algorithm

New inputs are tokenized closely following the BPE training process:

1. **Normalization**: The input text is normalized.
2. **Pre-tokenization**: The normalized text is split into words.
3. **Character Splitting**: Each word is split into individual characters.
4. **Applying Merge Rules**: The learned BPE merge rules are applied in sequence to the split characters.

### 🍎**Example**:
- **Merge Rules Learned**:
  - ("u", "g") → "ug"
  - ("u", "n") → "un"
  - ("h", "ug") → "hug"
  
- **Tokenization**:
  - "bug" → ["b", "ug"]
  - "mug" → ["[UNK]", "ug"] (since "m" is not in the base vocabulary)
  - "thug" → ["[UNK]", "hug"] (since "t" is not in the base vocabulary)

### A simplified BPE implementation

1. **Create a Corpus**: 
   - Example sentences are used to form a basic corpus:

In [1]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

2. **Pre-tokenization**:
   - Use a pre-trained tokenizer (like GPT-2) to split the corpus into words.

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/micromamba/envs/ml/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


3. **Compute Word Frequencies**:
   - Calculate the frequency of each word in the corpus.

In [3]:
from collections import defaultdict
word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


4. **Compute Base Vocabulary**:
   - Extract and sort all unique characters (alphabet) in the corpus.

In [4]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


   - Include special tokens like GPT-2’s `<|endoftext|>`:

In [5]:
vocab = ["<|endoftext|>"] + alphabet.copy()

5. **Split Words into Characters**:
   - Initialize word splits by breaking words into individual characters:

In [6]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

6. **Compute Pair Frequencies**:
   - Write a function to calculate the frequency of character pairs, which is essential for training the BPE merges:

In [7]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

   - Example output for pair frequencies:

In [8]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


- Find the most frequent pair

In [9]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


- The first merge to learn is ('Ġ', 't') -> 'Ġt'
  - add 'Ġt' to the vocabulary

In [10]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

- Apply that merge in the splits dictionary with another function

In [11]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

# The result of the first merge
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


- Loop until we have learned all the merges rules
  - suppose a vocab size of 50:

In [ ]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

- 19 merge rules learned 
   - the initial vocabulary had a size of 31 — 30 characters in the alphabet, 
   - plus the special token

In [13]:
print(len(merges), ":\n", merges)

19 :
 {('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}


- The vocabulary is composed of 
  - the special token, 
  - the initial alphabet, 
  - and all the results of the merges

In [14]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab', 'Ġtokeni']


- To tokenize a new text, 
  - we pre-tokenize it, split it, 
  - then apply all the merge rules learned

In [15]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

- Try this on any text composed of characters in the alphabet:

In [16]:
tokenize("This is natural language processing BPE algorithm.")

['This',
 'Ġis',
 'Ġ',
 'n',
 'a',
 't',
 'u',
 'r',
 'a',
 'l',
 'Ġ',
 'l',
 'a',
 'n',
 'g',
 'u',
 'a',
 'g',
 'e',
 'Ġ',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'in',
 'g',
 'Ġ',
 'B',
 'P',
 'E',
 'Ġa',
 'l',
 'g',
 'o',
 'r',
 'i',
 't',
 'h',
 'm',
 '.']

## Unigram Algorithm**

- Used in SentencePiece, applied in models like AlBERT, T5, mBART, Big Bird, and XLNet.
- Unlike BPE and WordPiece, Unigram starts with a large vocabulary and prunes it down.


### **Training Algorithm**

- Begins with a large vocabulary
  - Options for building initial vocabulary:
    - Use the most common substrings in pre-tokenized words.
    - Apply BPE with a large vocabulary size.
- Vocabulary Pruning
  - Computes loss for the corpus using the current vocabulary.
  - Calculates the impact on loss if each symbol is removed.
  - Removes the least impactful symbols until the desired vocabulary size is achieved.
    - Base characters are never removed to ensure any word can be tokenized.

---

### **Tokenization Process**
- **Concept**:
  - Unigram is a language model where each token is independent.
  - Probability of a token = (Frequency of the token) / (Sum of all frequencies in the vocabulary).
- 🍎 **Example**: 
  - Given the corpus: 
    - ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
  - Initial vocabulary of all strict substrings: 
    - ["h", "u", "g", "hu", "ug", "p", "pu", "n", "un", "b", "bu", "s", "hug", "gs", "ugs"]
  - Subword frequencies:
    - ("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16)
    - ("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)
  - ∴ the sum of all frequencies is 210
    - the probability of the subword "ug" is thus 20/210
    - the probability of the subword "ugs" is thus 5/210

---

### **Finding Optimal Tokenization**
- **Probability Computation**:
  - Given a word, `segment it into tokens in all the possible segmentations` 
    - compute the probability p of each according to the Unigram model
    - all tokens are considered independent, this probability is just the product of the probability of each token
- Tokenization with the highest probability is selected.
- 🍎 **Example**
  - Given word "pug", all its possible segmentations
    - p(['p','u','g']) = p('p') × p('u') × p('g') = (5/210)×(36/210)×(20/210) = 0.000389
    - p(['p','ug']) = p('p') × p('ug') = (5/210)×(20/210) = 0.0022676
    - p(['pu','g']) = p('pu') × p('g') = (5/210)×(20/210) = 0.0022676
  - Tokenizations with the least tokens possible will have the highest probability
    - Segmenations in a tie choose the first encountered

---

### **Viterbi Algorithm**
- **Purpose**:
  - Finds the most probable segmentation for a given word.
- **Process**:
  - Constructs a graph of possible segmentations.
  - Determines the best path through the graph, ending in the highest score.
- 🍎 **Example**
  - Given the previous vocabulary, for each position in `unhug`, the subwords with the `best scores ending there` are the following
    - Character 0 (u): "u" (score 0.171429)
    - Character 1 (n): "un" (score 0.076191)
    - Character 2 (h): "un" "h" (score 0.005442)
    - Character 3 (u): "un" "hu" (score 0.005442)
    - Character 4 (g): "un" "hug" (score 0.005442)
  - ∴  "unhug" would be tokenized as ["un", "hug"].

---

### **Back to Training**
- **Loss Computation**:
  - `Loss` is the negative log likelihood of tokenization scores across the corpus.
    - $\displaystyle 𝓁(corpus) = ∑_{w∈corpus} -f(w)\log(p(w))$
- 🍎 **Example**: Loss computation for the previous corpus `("hug", "pug", "pun", "bun", "hugs")`.
  - "hug": ["hug"] (score 0.071428)
  - "pug": ["pu", "g"] (score 0.007710)
  - "pun": ["pu", "n"] (score 0.006168)
  - "bun": ["bu", "n"] (score 0.001451)
  - "hugs": ["hug", "s"] (score 0.001701)
- ∴ loss = 10 * (-log(0.071428)) + 5 * (-log(0.007710)) + 12 * (-log(0.006168)) + 4 * (-log(0.001451)) + 5 * (-log(0.001701)) = 169.8
 
---

### **Example of Token Removal**
- **Impact on Loss**:
  - Demonstrates how removing tokens affects the loss.
  - Example: Removing "pu" vs. removing "hug".


